# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [1]:
# Imports here

import numpy as np
import torch
import matplotlib.pyplot as plt
from torchvision import datasets, transforms, models
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from collections import OrderedDict


## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [2]:
# TODO: Define your transforms for the training, validation, and testing sets# INDICAMOS LOS DIRECTORIO DE LOS 3 CONJUNTOS DE DATOS (/home/workspace/Cat_Dog_data/train)
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

# TODO: DEFINIMOS LAS NORMALIZACION DE DATOS PARA LOS 3 CONJUNTOS DE DATOS (TRAIN, VALIDATION Y TEST) INDICANDO LAS MEDIAS
# Y DESVIACIONES ESTANDAR PARA CADA UNO, ESTO LO HACEMOS PUES QUEREMOS LLEVAR LOS PIXELES A LOS RANGOS DE 0 A 1 USANDO
# LA DISTRIBUCION NORMAL

data_transforms = transforms.Compose([transforms.Resize([224,224]), transforms.CenterCrop([224,224]), 
                    transforms.ToTensor(), 
                    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])


#TRAEMOS LOS DATASET DE LOS TRES CONJUNTOS DE DATOS Y APLICAMOS LA TRANSFORMACION ANTERIOR
train_dataset = datasets.ImageFolder(train_dir, transform=data_transforms)
print(len(train_dataset))

valid_dataset = datasets.ImageFolder(valid_dir, transform=data_transforms)
print(len(valid_dataset))

test_dataset = datasets.ImageFolder(test_dir, transform=data_transforms)
print(len(test_dataset))

# TODO: DEFINIMOS LOS DATALOADER
dataloaders_train = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
dataloaders_valid = torch.utils.data.DataLoader(valid_dataset, batch_size=64, shuffle=True)
dataloaders_test = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=True)



6552
818
819


### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [3]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)
    print(cat_to_name)

{'21': 'fire lily', '3': 'canterbury bells', '45': 'bolero deep blue', '1': 'pink primrose', '34': 'mexican aster', '27': 'prince of wales feathers', '7': 'moon orchid', '16': 'globe-flower', '25': 'grape hyacinth', '26': 'corn poppy', '79': 'toad lily', '39': 'siam tulip', '24': 'red ginger', '67': 'spring crocus', '35': 'alpine sea holly', '32': 'garden phlox', '10': 'globe thistle', '6': 'tiger lily', '93': 'ball moss', '33': 'love in the mist', '9': 'monkshood', '102': 'blackberry lily', '14': 'spear thistle', '19': 'balloon flower', '100': 'blanket flower', '13': 'king protea', '49': 'oxeye daisy', '15': 'yellow iris', '61': 'cautleya spicata', '31': 'carnation', '64': 'silverbush', '68': 'bearded iris', '63': 'black-eyed susan', '69': 'windflower', '62': 'japanese anemone', '20': 'giant white arum lily', '38': 'great masterwort', '4': 'sweet pea', '86': 'tree mallow', '101': 'trumpet creeper', '42': 'daffodil', '22': 'pincushion flower', '2': 'hard-leaved pocket orchid', '54': 's

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

**Note for Workspace users:** If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [4]:
#PRECARGAMOS UNA RED YA EXISTENTE Y LA VEMOS
#OTRAS REDES https://pytorch.org/docs/master/torchvision/models.html
model = models.vgg16(pretrained=True)
model

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.torch/models/vgg16-397923af.pth
100%|██████████| 553433881/553433881 [00:10<00:00, 51637750.53it/s]


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

In [5]:

# HACEMOS UN FREEZE DE LOS PARAMETROS DE LA RED QUE TRAJIMOS Y CREAMOS UN NUEVO CLASIFICADOR PARA NO OCUPAR EL QUE TRAIA LA RED
for param in model.parameters():
    param.requires_grad = False

    model.classifier = nn.Sequential (nn.Linear(25088 ,1024) , 
                                 nn.ReLU() , 
                                 nn.Dropout(0.2) , 
                                 nn.Linear(1024 , 102) , 
                                 nn.LogSoftmax (dim = 1))
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

In [6]:
#CREAMOS UN OPTIMIZADOR Y LO PROBAMOS PARA VER SU FUNCIONAMIENTO

import time

for device in ['cpu', 'cuda']:

    criterion = nn.NLLLoss()
    # Only train the classifier parameters, feature parameters are frozen
    optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

    model.to(device)

    for ii, (inputs, labels) in enumerate(dataloaders_train):

        # Move input and label tensors to the GPU
        inputs, labels = inputs.to(device), labels.to(device)
        print('intputs are:', inputs)
        print('labels are:', labels)
        print('ii is:', ii)

        start = time.time()

        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if ii==3:
            break
        
    print(f"Device = {device}; Time per batch: {(time.time() - start)/3:.3f} seconds")

intputs are: tensor([[[[ 0.0398,  0.0227, -0.0287,  ...,  0.1939,  0.3138,  0.2967],
          [ 0.0398,  0.1083,  0.0398,  ...,  0.1426,  0.2111,  0.2282],
          [ 0.1083,  0.1254,  0.0741,  ...,  0.2111,  0.2111,  0.2282],
          ...,
          [-1.2788, -1.1418, -1.1247,  ..., -1.3302, -1.3987, -1.4672],
          [-1.3473, -1.2617, -1.2103,  ..., -1.3473, -1.4500, -1.5014],
          [-1.4158, -1.2788, -1.1760,  ..., -1.3473, -1.4672, -1.5185]],

         [[ 0.3978,  0.4153,  0.4153,  ...,  0.7129,  0.7479,  0.6429],
          [ 0.4328,  0.5203,  0.5028,  ...,  0.6429,  0.6254,  0.5728],
          [ 0.5378,  0.5903,  0.5728,  ...,  0.6779,  0.6254,  0.5728],
          ...,
          [-0.2325, -0.0399,  0.0301,  ..., -0.3550, -0.3901, -0.4076],
          [-0.3025, -0.1099,  0.0301,  ..., -0.3901, -0.3901, -0.3901],
          [-0.3550, -0.1099,  0.0826,  ..., -0.3901, -0.3725, -0.3725]],

         [[-0.4973, -0.5495, -0.5495,  ..., -0.2358, -0.0964, -0.1835],
          [-0.427

intputs are: tensor([[[[ 0.3481,  0.4166,  0.4166,  ...,  0.2796,  0.2967,  0.2796],
          [ 0.3138,  0.3994,  0.4166,  ...,  0.3138,  0.3309,  0.3138],
          [ 0.3138,  0.3652,  0.3994,  ...,  0.3652,  0.3994,  0.3652],
          ...,
          [ 0.8961,  0.9646,  0.9646,  ..., -1.9638, -1.8439, -1.0390],
          [ 0.9303,  0.9132,  0.9474,  ..., -1.9980, -1.9980, -1.8953],
          [ 0.8961,  0.8618,  0.9132,  ..., -1.9980, -1.9980, -1.9638]],

         [[ 0.4853,  0.5553,  0.5553,  ..., -1.2304, -1.2129, -1.2304],
          [ 0.4503,  0.5378,  0.5553,  ..., -1.1954, -1.1779, -1.1954],
          [ 0.4503,  0.5028,  0.5378,  ..., -1.1779, -1.1429, -1.1779],
          ...,
          [-0.9853, -0.9503, -1.0028,  ..., -1.8606, -1.8081, -1.1604],
          [-0.8803, -0.8978, -0.9153,  ..., -1.8782, -1.8957, -1.8431],
          [-0.9153, -0.9153, -0.8978,  ..., -1.8782, -1.9132, -1.8957]],

         [[ 0.6705,  0.7402,  0.7402,  ..., -1.0027, -0.9853, -1.0027],
          [ 0.635

intputs are: tensor([[[[-1.8097, -1.7925, -1.7754,  ..., -0.3883, -1.4500, -1.7240],
          [-1.8097, -1.8097, -1.8097,  ..., -0.9192, -1.6727, -1.7412],
          [-1.8097, -1.8097, -1.8268,  ..., -1.4672, -1.6727, -1.7412],
          ...,
          [-0.3883, -0.4054, -0.5253,  ..., -0.5938, -1.4843, -1.8782],
          [-0.3369, -0.3883, -0.4911,  ..., -0.6623, -1.6555, -1.9124],
          [-0.3369, -0.3883, -0.4911,  ..., -0.8164, -1.8439, -1.9124]],

         [[-1.3529, -1.3354, -1.3179,  ...,  0.6954, -0.5826, -1.0203],
          [-1.3704, -1.3704, -1.3529,  ...,  0.1352, -0.8277, -1.0378],
          [-1.3704, -1.3704, -1.3880,  ..., -0.5126, -0.8803, -1.0553],
          ...,
          [ 1.0805,  1.0805,  0.9230,  ...,  0.2752, -0.8452, -1.3880],
          [ 1.1331,  1.0805,  1.0105,  ...,  0.1877, -1.0203, -1.4230],
          [ 1.1155,  1.0805,  1.0280,  ...,  0.0301, -1.2129, -1.4230]],

         [[-1.6650, -1.6476, -1.6302,  ..., -0.5495, -1.3861, -1.5604],
          [-1.647

intputs are: tensor([[[[ 0.4166,  1.0159,  1.2899,  ...,  1.9407,  1.9578,  1.9749],
          [ 0.4508,  1.0502,  1.2728,  ...,  1.9578,  1.9578,  1.9749],
          [ 0.5364,  1.0331,  1.1187,  ...,  1.9749,  1.9578,  1.9749],
          ...,
          [ 0.0912, -0.0287,  0.2967,  ...,  1.4098,  1.3242,  1.2557],
          [ 0.1083,  0.2111,  0.4679,  ...,  1.4269,  1.3584,  1.3070],
          [ 0.1426,  0.3823,  0.5878,  ...,  1.4269,  1.3927,  1.3584]],

         [[ 0.0126,  0.4503,  0.5553,  ...,  2.1134,  2.1310,  2.1485],
          [ 0.1527,  0.5903,  0.6779,  ...,  2.1310,  2.1310,  2.1485],
          [ 0.3627,  0.7654,  0.7654,  ...,  2.1485,  2.1310,  2.1485],
          ...,
          [-1.5980, -1.7906, -1.4930,  ..., -0.7577, -0.8277, -0.8978],
          [-1.8431, -1.8081, -1.4055,  ..., -0.7402, -0.7927, -0.8277],
          [-1.8256, -1.6856, -1.3004,  ..., -0.7577, -0.7577, -0.7577]],

         [[ 0.3742,  0.5485,  0.2696,  ...,  2.2914,  2.3088,  2.3263],
          [ 0.321

Device = cpu; Time per batch: 12.970 seconds
intputs are: tensor([[[[-1.1247, -1.0733, -0.9363,  ...,  1.2043,  1.2557,  1.3413],
          [-0.8678, -0.7479, -0.5596,  ...,  1.3927,  1.2557,  1.2899],
          [-0.4739, -0.2684, -0.0116,  ...,  1.6495,  1.3242,  1.2728],
          ...,
          [ 0.6734,  0.3481, -0.1314,  ..., -1.6384, -1.6213, -1.6042],
          [ 0.4679,  0.0227, -0.5596,  ..., -1.6727, -1.6213, -1.6384],
          [ 0.2796, -0.2342, -0.8507,  ..., -1.7069, -1.6555, -1.6727]],

         [[-1.8957, -1.8957, -1.8782,  ..., -0.9678, -1.0553, -1.0728],
          [-1.7731, -1.7031, -1.6331,  ..., -0.7577, -1.0203, -1.1078],
          [-1.5980, -1.4405, -1.2829,  ..., -0.5126, -0.9328, -1.1078],
          ...,
          [-0.2325, -0.5126, -0.8277,  ..., -1.8782, -1.8606, -1.8431],
          [-0.4776, -0.7577, -1.0728,  ..., -1.9657, -1.9482, -1.9482],
          [-0.6702, -1.0028, -1.3354,  ..., -2.0007, -1.9832, -2.0007]],

         [[-1.1247, -1.0898, -0.9504,  ..., 

intputs are: tensor([[[[-2.1008, -2.0665, -2.0837,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1008, -2.0665, -2.1008,  ..., -2.1179, -2.1179, -2.1179],
          [-2.0837, -2.0665, -2.1008,  ..., -2.1179, -2.1179, -2.1179],
          ...,
          [-2.0837, -2.0837, -2.0837,  ..., -1.4843, -1.5014, -1.5014],
          [-2.1008, -2.1008, -2.0837,  ..., -1.5357, -1.5014, -1.4329],
          [-2.1179, -2.1008, -2.1008,  ..., -1.4672, -1.4672, -1.4158]],

         [[-1.9482, -1.9132, -1.9307,  ..., -2.0357, -2.0357, -2.0357],
          [-1.9482, -1.9132, -1.9482,  ..., -2.0357, -2.0357, -2.0357],
          [-1.9307, -1.9132, -1.9482,  ..., -2.0357, -2.0357, -2.0357],
          ...,
          [-1.9307, -1.9132, -1.8782,  ..., -0.7752, -0.7927, -0.7752],
          [-1.8256, -1.8606, -1.8957,  ..., -0.7402, -0.7227, -0.6527],
          [-1.8081, -1.8431, -1.8957,  ..., -0.6527, -0.6702, -0.6176]],

         [[-1.6650, -1.6302, -1.6476,  ..., -1.8044, -1.8044, -1.8044],
          [-1.665

intputs are: tensor([[[[-0.1486, -0.3369, -0.5424,  ..., -0.4226, -0.4054, -0.4054],
          [-0.1486, -0.3198, -0.5596,  ..., -0.4226, -0.4054, -0.4054],
          [-0.1486, -0.3198, -0.5938,  ..., -0.4226, -0.4054, -0.4054],
          ...,
          [ 0.7591,  0.8104,  0.7933,  ..., -0.8849, -0.8678, -0.8678],
          [ 0.6734,  0.7077,  0.7077,  ..., -0.9877, -0.9534, -0.9877],
          [ 0.5364,  0.5364,  0.5022,  ..., -1.0733, -1.0562, -1.0904]],

         [[-0.4426, -0.6001, -0.7752,  ...,  0.4153,  0.4153,  0.4153],
          [-0.4251, -0.5826, -0.7927,  ...,  0.4153,  0.4153,  0.4153],
          [-0.4251, -0.5826, -0.8277,  ...,  0.4153,  0.4153,  0.4153],
          ...,
          [ 0.7654,  0.8179,  0.8004,  ..., -1.1253, -1.0903, -1.0903],
          [ 0.5378,  0.5903,  0.5903,  ..., -1.1954, -1.1779, -1.1954],
          [ 0.3627,  0.3803,  0.3452,  ..., -1.2829, -1.2654, -1.3004]],

         [[-0.4275, -0.6541, -0.8458,  ..., -1.3339, -1.3687, -1.3687],
          [-0.427

intputs are: tensor([[[[-2.0152, -2.0665, -2.1008,  ..., -0.8849, -0.9192, -0.9705],
          [-2.0837, -2.1008, -2.1008,  ..., -0.9705, -0.9705, -1.0048],
          [-2.1008, -2.1008, -2.0665,  ..., -0.9705, -0.9705, -0.9877],
          ...,
          [-2.1179, -2.1179, -2.1179,  ..., -1.9467, -2.0837, -2.1008],
          [-2.1179, -2.1179, -2.1179,  ..., -1.9467, -2.0837, -2.1008],
          [-2.1179, -2.1179, -2.1179,  ..., -1.8953, -2.0837, -2.1008]],

         [[-1.6506, -1.7731, -1.9132,  ...,  0.4503,  0.4153,  0.3627],
          [-1.8081, -1.8957, -2.0007,  ...,  0.3627,  0.3627,  0.3277],
          [-1.9482, -2.0007, -2.0357,  ...,  0.3627,  0.3627,  0.3452],
          ...,
          [-2.0357, -2.0357, -2.0357,  ..., -1.9657, -2.0182, -2.0182],
          [-2.0357, -2.0357, -2.0357,  ..., -1.9832, -2.0182, -2.0182],
          [-2.0357, -2.0357, -2.0357,  ..., -1.9132, -2.0182, -2.0182]],

         [[-1.7347, -1.7696, -1.7870,  ..., -1.0027, -1.0724, -1.1421],
          [-1.787

In [8]:
#CREAMOS UN VALIDADOR LOS RESULTADOS DE NUESTRA RED PARA OCUPARLO ABAJO EN EL ENTRENAMIENTO

#ENVIAMOS LOS INPUTS, LOS LABELS Y LOS MODELS AL DEVICE QUE ESTEMOS OCUPANDO PARA NO TENER ERROR CUANDO OCUPAMOS CUDA EN EL GPU
inputs, labels = data
inputs, labels = inputs.cuda(), labels.cuda()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
inputs, labels = inputs.to(device), labels.to(device)
model = model.to(device)
print(device)

criterion = nn.NLLLoss()
print(criterion)
optimizer = optim.Adam(model.classifier.parameters(), lr = 0.001)
print(optimizer)

def valid_cal_accuracy(model, dataloaders_valid):
    
    model.eval()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
    
    total = 0
    valid_loss = 0
    valid_accuracy = 0
    with torch.no_grad():
        for data in dataloaders_valid:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            valid_loss += loss.item()
            
            total += labels.size(0)
            valid_accuracy += (predicted == labels).sum().item() / total
    return valid_loss, valid_accuracy

#ENTRENAMOS NUESTRA RED

epochs = 6
steps = 0
print_every = 100

for epoch in range(epochs):

    model.train()
    running_loss = 0

    for ii, (inputs, labels) in enumerate(dataloaders_train):
        steps += 1
        # Move input and label tensors to the default-device 
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(inputs)     

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if steps % print_every == 0:
            valid_loss, valid_accuracy = valid_cal_accuracy(model, dataloaders_valid)
            print(f"Epoch {epoch+1}/{epochs}.. "
                f"Train loss: {running_loss/ len(dataloaders_train):.3f}.. " 
                f"Validation loss: {valid_loss/len(dataloaders_valid):.3f}.. "
                f"Validation accuracy: {valid_accuracy:.3f}")
                
            running_loss = 0
            

cuda:0
NLLLoss()
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)
Epoch 1/6.. Train loss: 1.345.. Validation loss: 0.581.. Validation accuracy: 2.706
Epoch 2/6.. Train loss: 0.106.. Validation loss: 0.457.. Validation accuracy: 2.691
Epoch 3/6.. Train loss: 0.024.. Validation loss: 0.383.. Validation accuracy: 2.820
Epoch 4/6.. Train loss: 0.007.. Validation loss: 0.384.. Validation accuracy: 2.861
Epoch 5/6.. Train loss: 0.005.. Validation loss: 0.368.. Validation accuracy: 2.930
Epoch 6/6.. Train loss: 0.007.. Validation loss: 0.349.. Validation accuracy: 2.874


## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [7]:
# TESTEAMOS NUESTRA RED

#ENVIAMOS LOS INPUTS, LOS LABELS Y LOS MODELS AL DEVICE QUE ESTEMOS OCUPANDO PARA NO TENER ERROR CUANDO OCUPAMOS CUDA EN EL GPU
inputs, labels = inputs.cuda(), labels.cuda()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
inputs, labels = inputs.to(device), labels.to(device)
model = model.to(device)

print(device)

model.eval()
test_accuracy = 0

with torch.no_grad():
    for data in dataloaders_test: 
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model.forward(images)
        ps = torch.exp(outputs)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        test_accuracy += torch.mean(equals.type(torch.FloatTensor))

print('La precisión del testeo de nuestra red es: %d %%' % (test_accuracy))

cuda:0
La precisión del testeo de nuestra red es: 2 %


## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
# TODO: Save the checkpoint 

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# TODO: Write a function that loads a checkpoint and rebuilds the model

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    # TODO: Process a PIL image for use in a PyTorch model

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes